# Import

In [1]:
import networkx as nx

In [2]:
from tqdm import tqdm

In [3]:
with open('3.txt') as f:
    lines = f.readlines()

# Description of the data

In [4]:
len(lines)

20005

In [5]:
lines[0] #the number of vertices in the phenotype tree

'100000\n'

In [6]:
n = int(lines[0].rstrip())
n

100000

In [7]:
len(lines[1]) #the parent identifiers for the vertices

581244

In [8]:
parents = [int(i) for i in lines[1].rstrip().split(' ')]

In [9]:
parents[:5]

[1, 2, 3, 4, 5]

In [10]:
len(lines[2]) #information content values of the corresponding vertices

855632

In [11]:
info = [int(i) for i in lines[2].rstrip().split(' ')]

In [12]:
info[:5]

[784, 1499, 2436, 3329, 4162]

In [13]:
lines[3] #the number of diseases

'10000\n'

In [14]:
m = int(lines[3].rstrip())
m

10000

In [15]:
len(lines[4:m+4]) #contain descriptions of diseases

10000

In [16]:
lines[4],lines[m+4]

('6 80127 45276 14858 88937 85190 72807\n', '10000\n')

In [17]:
desc_d = []
for line in lines[4:m+4]:
    desc_d.append([int(i) for i in line.rstrip().split(' ')])

In [18]:
desc_d[-1]

[1, 8591]

In [19]:
lines[m+4] #the number of patients

'10000\n'

In [20]:
l = int(lines[m+4].rstrip())
l

10000

In [21]:
lines[m+4+1]

'6 49032 67424 45587 27267 5181 51174\n'

In [22]:
len(lines[m+4+1:m+4+1+l]) #descriptions of patients

10000

In [23]:
desc_p = []
for line in lines[m+4+1:m+4+1+l]:
    desc_p.append([int(i) for i in line.rstrip().split(' ')])

In [24]:
desc_p[-1]

[3, 84123, 83900, 61365]

In [25]:
lines[m+4+l]

'3 84123 83900 61365\n'

# Create Tree Structure

## Create Empty Graph Object

In [26]:
# create empty graph object
# must use directed graph in order to use LCA algorithmn in NX
G = nx.DiGraph()

## Create Tree

In [27]:
# initialize root
G.add_node(1)

In [28]:
%%time
# create edges
edges = []
for i, j in zip(parents,range(2,n+1)):
    edges.append((i,j))

CPU times: user 56.6 ms, sys: 19.4 ms, total: 76 ms
Wall time: 76 ms


In [29]:
%%time
# create tree graph
G.add_edges_from(edges)

CPU times: user 1.12 s, sys: 152 ms, total: 1.27 s
Wall time: 1.27 s


In [30]:
import gc
gc.collect()

22

# Task

## IC dictionary

In [31]:
# create info dictionary
infod = {}

In [32]:
%%time
for i , j in enumerate(info):
    infod[i+1] = j

CPU times: user 41 ms, sys: 30.3 ms, total: 71.3 ms
Wall time: 70.5 ms


# Other variables

In [49]:
# disease score
d_scores = []
# result output
result = []
# create score
score = 0
# create minimum
maxim = 0

In [50]:
gc.collect()

96

# Main code block

In [44]:
# function generator to optimize loop
def product(a, b):
    for i in a:
        for j in b:
            yield i, j

In [51]:
%%time 

# SKIP IC calculation
# Use generator

# loop through patients
for i in tqdm(range(1,l+1)):
    d_scores = []
    # loop through diseases
    for j in range(1, m+1):
        score = 0
        for d, q in product(desc_p[i-1][1:], desc_d[j-1][1:]):
            lca = nx.tree_all_pairs_lowest_common_ancestor(G, root=1, pairs=[(d,q)])
            anc = next(lca)[-1]
            score += anc
        d_scores.append(score)
    disease = d_scores.index(max(d_scores)) +1
    result.append(str(disease) + '\n')

  0%|          | 0/10000 [03:43<?, ?it/s]


KeyboardInterrupt: 

In [38]:
with open('result3.txt', 'w') as f:
    f.writelines(result)

In [39]:
len(result)

0

## still slow, completely skip IC calculation, generator for d&p, optimized LCA algorithm from networkx

## Still way too long with networkx, 9 mins+ for one patient, need optimization on loops, try product generator?**

## Too Slow, use Networkx Package in Python instead of AnyTree